Download Model from HuggingFace

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
from huggingface_hub import snapshot_download

model_name = "meta-llama/Meta-Llama-3.1-8B"
base_model = "./original_model/"
snapshot_download(repo_id=model_name, local_dir=base_model, ignore_patterns=["*.pth"])

Clone llama.cpp Repository

In [ ]:
!git clone https://github.com/ggerganov/llama.cpp

Cloning into 'llama.cpp'...
remote: Enumerating objects: 31643, done.
remote: Counting objects: 100% (8922/8922), done.
remote: Compressing objects: 100% (611/611), done.
remote: Total 31643 (delta 8622), reused 8404 (delta 8303), pack-reused 22721
Receiving objects: 100% (31643/31643), 55.35 MiB | 18.09 MiB/s, done.
Resolving deltas: 100% (22715/22715), done.


In [ ]:
!mkdir models

Convert model to GGUF format

In [ ]:
!python llama.cpp/convert_hf_to_gguf.py ./original_model/ --outfile models/llama_3.1_FP16.gguf

INFO:hf-to-gguf:Loading model: original_model
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:gguf: loading model weight map from 'model.safetensors.index.json'
INFO:hf-to-gguf:gguf: loading model part 'model-00001-of-00004.safetensors'
INFO:hf-to-gguf:token_embd.weight,           torch.bfloat16 --> F16, shape = {4096, 128256}
INFO:hf-to-gguf:blk.0.attn_norm.weight,      torch.bfloat16 --> F32, shape = {4096}
INFO:hf-to-gguf:blk.0.ffn_down.weight,       torch.bfloat16 --> F16, shape = {14336, 4096}
INFO:hf-to-gguf:blk.0.ffn_gate.weight,       torch.bfloat16 --> F16, shape = {4096, 14336}
INFO:hf-to-gguf:blk.0.ffn_up.weight,         torch.bfloat16 --> F16, shape = {4096, 14336}
INFO:hf-to-gguf:blk.0.ffn_norm.weight,       torch.bfloat16 --> F32, shape = {4096}
INFO:hf-to-gguf:blk.0.attn_k.weight,         torch.bfloat16 --> F16, shape = {4096, 1024}
INFO:hf-to-gguf:blk.0.attn_output.weight,    torch.bfloat16 --> F16,

Build llama.cpp and quantize the Model

In [ ]:
!mkdir llama.cpp/build && cd llama.cpp/build && cmake .. && cmake --build . --config Release

-- The C compiler identification is GNU 11.4.0
-- The CXX compiler identification is GNU 11.4.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Found Git: /usr/bin/git (found version "2.34.1")
-- Performing Test CMAKE_HAVE_LIBC_PTHREAD
-- Performing Test CMAKE_HAVE_LIBC_PTHREAD - Success
-- Found Threads: TRUE
-- Found OpenMP_C: -fopenmp (found version "4.5")
-- Found OpenMP_CXX: -fopenmp (found version "4.5")
-- Found OpenMP: TRUE (found version "4.5")
-- OpenMP found
-- Using llamafile
-- Warning: ccache not found - consider installing it for faster compilation or disable this warning with GGML_CCACHE=OFF
-- CMAKE_SYSTEM_PROCE

In [ ]:
!cd llama.cpp/build/bin && ./llama-quantize /content/models/llama_3.1_FP16.gguf /content/models/llama_3.1-Q4_K_M.gguf q4_K_M

main: build = 3543 (ebd541a5)
main: built with cc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0 for x86_64-linux-gnu
main: quantizing '/content/models/llama_3.1_FP16.gguf' to '/content/models/llama_3.1-Q4_K_M.gguf' as Q4_K_M
llama_model_loader: loaded meta data with 26 key-value pairs and 292 tensors from /content/models/llama_3.1_FP16.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Original_Model
llama_model_loader: - kv   3:                         general.size_label str              = 8.0B
llama_model_loader: - kv   4:                            general.license str              = llama3.1
llama_model_loader: - kv   5:          

Inference using Quantized Model

In [ ]:
!pip install llama-cpp-python==0.2.85

  Using cached llama_cpp_python-0.2.85.tar.gz (49.3 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached diskcache-5.6.3-py3-none-any.whl.metadata (20 kB)
Using cached diskcache-5.6.3-py3-none-any.whl (45 kB)
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.2.85-cp310-cp310-linux_x86_64.whl size=2857609 sha256=c33d5417abce7c6e5aba06e453b1f6e337bf190c929e8de280e24acbbc55d634
  Stored in directory: /root/.cache/pip/wheels/3f/e8/4e/29a754f9175ef52b6481cd75e3af4de38bf6dfa9c2972f75d4
Successfully built llama-cpp-python


In [ ]:
from llama_cpp import Llama

In [ ]:
model_path = "/content/models/llama_3.1-Q4_K_M.gguf"

In [ ]:
llm = Llama(model_path=model_path)

llama_model_loader: loaded meta data with 26 key-value pairs and 292 tensors from /content/models/llama_3.1-Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Original_Model
llama_model_loader: - kv   3:                         general.size_label str              = 8.0B
llama_model_loader: - kv   4:                            general.license str              = llama3.1
llama_model_loader: - kv   5:                               general.tags arr[str,6]       = ["facebook", "meta", "pytorch", "llam...
llama_model_loader: - kv   6:                          general.languages arr[str,8]       = ["en", "de", "fr", "it", "pt"

Test on text only tasks

In [ ]:
generation_kwargs = {
    "max_tokens":200,
    "echo":False,
    "top_k":1
}

prompt = "Which country hosted 2018 fifa world cup?"
res = llm(prompt, **generation_kwargs)
res


llama_print_timings:        load time =    5582.23 ms
llama_print_timings:      sample time =      22.08 ms /   200 runs   (    0.11 ms per token,  9056.33 tokens per second)
llama_print_timings: prompt eval time =    5582.13 ms /    12 tokens (  465.18 ms per token,     2.15 tokens per second)
llama_print_timings:        eval time =  152621.22 ms /   199 runs   (  766.94 ms per token,     1.30 tokens per second)
llama_print_timings:       total time =  158636.78 ms /   211 tokens


{'id': 'cmpl-edb6eac3-82b3-4880-91d8-30c17069b277',
 'object': 'text_completion',
 'created': 1723112699,
 'model': '/content/models/llama_3.1-Q4_K_M.gguf',
 'choices': [{'text': " The 2018 fifa world cup was the 21st fifa world cup, an international football tournament contested by the men's national teams of the member associations of fifa. It took place in russia from 14 june to 15 july 2018. It was the first world cup to be held in eastern europe, and the 11th time that it had been held in europe. At an estimated cost of over $14.2 billion, it. The 2018 fifa world cup was the 21st fifa world cup, an international football tournament contested by the men's national teams of the member associations of fifa. It took place in russia from 14 june to 15 july 2018. It was the first world cup to be held in eastern europe, and the 11th time that it had been held in europe. At an estimated cost of over $14.2 billion, it. The 2018",
   'index': 0,
   'logprobs': None,
   'finish_reason': 'len

Save model to Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!mkdir "/content/drive/My Drive/llama_models"

In [ ]:
import shutil

source_file_path = '/content/models/llama_3.1-Q4_K_M.gguf'
destination_file_path = '/content/drive/My Drive/llama_models/llama_3.1-Q4_K_M.gguf'

shutil.copy(source_file_path, destination_file_path)

Upload model to Huggingface Hub

In [ ]:
from huggingface_hub import login
login('your_token')

In [ ]:
from huggingface_hub import HfApi
api = HfApi()

model_id = "llama3.1-Q4_K_M-gguf"
api.create_repo(model_id, exist_ok=True, repo_type="model")
api.upload_file(
    path_or_fileobj='/content/models/llama_3.1-Q4_K_M.gguf',
    path_in_repo="llama3.1-Q4_K_M.gguf",
    repo_id=model_id,
)

Inference by utilizing GPU

In [ ]:
from huggingface_hub import snapshot_download

model_name = "llama3.1-Q4_K_M-gguf"
base_model = "./quantized_models/"
snapshot_download(repo_id=model_name, local_dir=base_model)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

.gitattributes:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

llama3.1-Q4_K_M.gguf:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

'/content/quantized_models'

In [ ]:
!pip install llama-cpp-python --extra-index-url https://abetlen.github.io/llama-cpp-python/whl/cu122

Looking in indexes: https://pypi.org/simple, https://abetlen.github.io/llama-cpp-python/whl/cu122
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 394.5/394.5 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.6 MB/s eta 0:00:00


In [ ]:
from llama_cpp import Llama
model_path = "./quantized_models/llama3.1-Q4_K_M.gguf"

model = Llama(model_path=model_path, n_gpu_layers=-1)

llama_model_loader: loaded meta data with 26 key-value pairs and 292 tensors from ./quantized_models/llama3.1-Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Original_Model
llama_model_loader: - kv   3:                         general.size_label str              = 8.0B
llama_model_loader: - kv   4:                            general.license str              = llama3.1
llama_model_loader: - kv   5:                               general.tags arr[str,6]       = ["facebook", "meta", "pytorch", "llam...
llama_model_loader: - kv   6:                          general.languages arr[str,8]       = ["en", "de", "fr", "it", "p

In [ ]:
generation_kwargs = {
    "max_tokens":200,
    "echo":False,
    "top_k":1
}

prompt = "Which country hosted 2018 fifa world cup?"
res = model(prompt, **generation_kwargs)
res

Llama.generate: prefix-match hit

llama_print_timings:        load time =     545.44 ms
llama_print_timings:      sample time =      16.89 ms /   200 runs   (    0.08 ms per token, 11842.03 tokens per second)
llama_print_timings: prompt eval time =      67.41 ms /     9 tokens (    7.49 ms per token,   133.52 tokens per second)
llama_print_timings:        eval time =    5408.86 ms /   199 runs   (   27.18 ms per token,    36.79 tokens per second)
llama_print_timings:       total time =    5691.39 ms /   208 tokens


{'id': 'cmpl-94cf14c4-8b6e-4861-a2c8-0eec891a92c2',
 'object': 'text_completion',
 'created': 1723116609,
 'model': './quantized_models/llama3.1-Q4_K_M.gguf',
 'choices': [{'text': " The 2018 fifa world cup was the 21st fifa world cup, an international football tournament contested by the men's national teams of the member associations of fifa. It took place in russia from 14 june to 15 july 2018. It was the first world cup to be held in eastern europe, and the 11th time that it had been held in europe. At an estimated cost of over $14.2 billion, it. The 2018 fifa world cup was the 21st fifa world cup, an international football tournament contested by the men's national teams of the member associations of fifa. It took place in russia from 14 june to 15 july 2018. It was the first world cup to be held in eastern europe, and the 11th time that it had been held in europe. At an estimated cost of over $14.2 billion, it. The 2018",
   'index': 0,
   'logprobs': None,
   'finish_reason': 'l

In [ ]:
output = model("Provide Information about world war 2 in 1000 words.", max_tokens=2048, stop=["\n"], echo=False)

In [ ]:
print(output['choices'][0]['text'])

Loading Quantized Model from HuggingFace

In [ ]:
!pip install -U llama-cpp-python
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
# !pip install llama-cpp-python

from llama_cpp import Llama

llm = Llama.from_pretrained(
	repo_id="lmstudio-community/MiniCPM-V-2_6-GGUF",
	filename="MiniCPM-V-2_6-Q4_K_M.gguf",
)

Prompt Testing

In [ ]:
llm.create_chat_completion(
	messages = [
		{
			"role": "user",
			"content": [
				{
					"type": "text",
					"text": "Describe this image in one sentence."
				},
				{
					"type": "image_url",
					"image_url": {
						"url": "https://cdn.britannica.com/61/93061-050-99147DCE/Statue-of-Liberty-Island-New-York-Bay.jpg"
					}
				}
			]
		}
	]
)

Loading COCO captions dataset

In [ ]:
!pip install -q pillow tqdm evaluate requests
import os, json
from PIL import Image
from tqdm import tqdm

# Create folders
os.makedirs("coco_images", exist_ok=True)
os.makedirs("coco_ann", exist_ok=True)

# Load captions JSON (local file)
with open("coco_ann/annotations/captions_val2017.json") as f:
    data = json.load(f)

# Build a small local dataset (100 captions)
subset = data["annotations"][:100]

subset


In [ ]:
samples = []
for ann in tqdm(subset):
    img_id = ann["image_id"]
    img_url = f"http://images.cocodataset.org/val2017/{img_id:012d}.jpg"
    img_path = f"coco_images/{img_id:012d}.jpg"

    if not os.path.exists(img_path):
        r = requests.get(img_url, stream=True)
        if r.status_code == 200:
            with open(img_path, "wb") as f:
                f.write(r.content)

    samples.append({
        "image_path": img_path,
        "caption": ann["caption"]
    })

print("✅ Downloaded", len(samples), "samples.")


Prompt Template

In [ ]:
def get_caption(image_path):
    res = llm.create_chat_completion(
        messages=[{
            "role": "user",
            "content": [
                {"type": "text", "text": "Describe this image in one sentence."},
                {"type": "image_url", "image_url": {"url": image_path}}
            ]
        }]
    )
    return res["choices"][0]["message"]["content"]


Evaluation

In [ ]:
!pip install nltk rouge_score

Evaluation code 1

In [ ]:
import nltk
from nltk.translate.bleu_score import corpus_bleu, SmoothingFunction
from evaluate import load
from tqdm import tqdm
import time

nltk.download('punkt')

bleu_eval = load("bleu")
meteor_eval = load("meteor")
rouge_eval = load("rouge")

preds, refs, latencies = [], [], []
smooth = SmoothingFunction().method1

for s in tqdm(samples):
    start = time.time()
    pred = get_caption(s["image_path"])
    end = time.time()

    preds.append(pred)
    refs.append(s["caption"])
    latencies.append(end - start)

# --- Text normalization for fairer scoring ---
def normalize(text):
    return text.strip().lower().replace('.', '').replace(',', '')

norm_preds = [normalize(p) for p in preds]
norm_refs = [[normalize(r)] for r in refs]   # list of lists for corpus_bleu

# --- Smoothed BLEU ---
bleu_score = corpus_bleu(norm_refs, [p.split() for p in norm_preds],
                         smoothing_function=smooth)

# --- METEOR & ROUGE from evaluate ---
meteor_score = meteor_eval.compute(predictions=norm_preds, references=[r[0] for r in norm_refs])["meteor"]
rouge_score = rouge_eval.compute(predictions=norm_preds, references=[r[0] for r in norm_refs])["rougeL"]

print("\n=== Evaluation Summary ===")
print(f"Images evaluated: {len(samples)}")
print(f"Avg Latency: {sum(latencies)/len(latencies):.2f} s")
print(f"BLEU (smoothed): {bleu_score:.4f}")
print(f"METEOR: {meteor_score:.4f}")
print(f"ROUGE-L: {rouge_score:.4f}")


Evaluation Code 2

In [ ]:
import os
import json
import time
import nltk
import torch
from tqdm import tqdm
from collections import defaultdict

# BLEU scorer
from nltk.translate.bleu_score import corpus_bleu, SmoothingFunction

# HF evaluate metrics
from evaluate import load


# -------------------------------------------
# 1. Setup
# -------------------------------------------
nltk.download('punkt')
smooth = SmoothingFunction().method1

meteor_eval = load("meteor")
rouge_eval = load("rouge")
bertscore_eval = load("bertscore")


# -------------------------------------------
# 2. Load COCO annotations (local JSON)
# -------------------------------------------
with open("coco_ann/annotations/captions_val2017.json", "r") as f:
    data = json.load(f)

# -------------------------------------------
# 3. Build mapping: image_id → list of 5 captions
# -------------------------------------------
image_to_refs = defaultdict(list)
for ann in data["annotations"]:
    image_to_refs[ann["image_id"]].append(ann["caption"])

# -------------------------------------------
# 4. Collect predictions + references
# -------------------------------------------
preds, refs, latencies = [], [], []

def normalize(text):
    return text.strip().lower()

for s in tqdm(samples):
    img_path = s["image_path"]
    img_id = int(os.path.basename(img_path).split(".")[0])

    # Generate caption
    start = time.time()
    pred = get_caption(img_path)
    end = time.time()

    preds.append(normalize(pred))
    latencies.append(end - start)

    refs.append([normalize(r) for r in image_to_refs[img_id]])

# -------------------------------------------
# 5. Tokenize for BLEU
# -------------------------------------------
tok_preds = [nltk.word_tokenize(p) for p in preds]
tok_refs = [[nltk.word_tokenize(r) for r in rlist] for rlist in refs]

# -------------------------------------------
# 6. BLEU-4 (correct COCO scoring)
# -------------------------------------------
bleu4 = corpus_bleu(
    tok_refs, tok_preds,
    smoothing_function=smooth,
    weights=(0.25, 0.25, 0.25, 0.25)
)

# -------------------------------------------
# 7. METEOR & ROUGE
# -------------------------------------------
meteor = meteor_eval.compute(
    predictions=preds,
    references=refs
)["meteor"]

rouge_l = rouge_eval.compute(
    predictions=preds,
    references=[rlist[0] for rlist in refs]
)["rougeL"]


# -------------------------------------------
# 10. BERTScore (semantic embedding)
# -------------------------------------------
bertscore_f1 = bertscore_eval.compute(
    predictions=preds,
    references=[rlist[0] for rlist in refs],
    model_type="bert-base-uncased"
)["f1"]
bertscore_f1 = sum(bertscore_f1) / len(bertscore_f1)

# -------------------------------------------
# 11. CLIPScore (image–text alignment)
# -------------------------------------------


# -------------------------------------------
# 12. Print results
# -------------------------------------------
print("\n===== FINAL EVALUATION REPORT =====")
print(f"Images Evaluated:   {len(samples)}")
print(f"Avg Latency:        {sum(latencies)/len(latencies):.3f} sec\n")

print(f"BLEU-4:             {bleu4:.4f}")
print(f"METEOR:             {meteor:.4f}")
print(f"ROUGE-L:            {rouge_l:.4f}")
print(f"BERTScore (F1):     {bertscore_f1:.4f}")

